In [16]:
import pandas as pd
import numpy as np
from requests import get
from bs4 import BeautifulSoup
import contractions
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import nltk
from unidecode import unidecode
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
### assign headers since IMDB rejects the requests without it
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.50'}

In [3]:
## load static URL list (from HW5)

url_list = pd.read_csv("https://raw.githubusercontent.com/boneeyah/DS7337/main/mb_hw5_urls.csv")

In [4]:
# function goes through the table with the URLs to get each direct URL
# Parses through the content of each URL to grab the main review
# tokenizes the sentences of each review
# returns a dataframe with the movie title, review id, and the setence tokens
def grab_review(links_table):
    text = []
    for i in range(len(links_table)):
        review = get(links_table.url[i],headers)
        review_soup = BeautifulSoup(review.content, 'html.parser')
        text.append(review_soup.find(class_='text show-more__control').text)
    return(pd.DataFrame({'movie':links_table.movie,
                         'review':links_table.review,
                         'text':text                         
                        }))

In [5]:
review_text = grab_review(url_list)

In [6]:
special = ['\x96',':',',','-','(',')','[',']','–','/','#','``',';','.','&','"',"''",'?','!','....','--','...','*','..',"'"]
stop_words = nltk.corpus.stopwords.words('english') + ['movie','film','horror', 'thing','quiet','place','alien','covenant','shining','films','john','one']
special = stop_words + ["'s","'t","'d","'ll","'m","'re","'ve","n't"] + special

In [7]:
def normalize_list(review_list):
    expanded_words = [contractions.fix(term) for term in review_list] #expands contractions
    term = [word_tokenize(term.lower()) for term in expanded_words] #tokenizes the expanded strings
    blank_list = []
    for i in range(len(review_list)):
        blank_list.append(' '.join([unidecode(w) for w in term[i] if w not in special and w.isalpha()])) #removes special characters numbers, puts string back together
    return(np.array(blank_list))

In [23]:
review_text['norm_text'] = normalize_list(review_text.text)

In [51]:
def sent_analyzer(review,threshold,verbose):
    analyzer = SentimentIntensityAnalyzer()
    pol_scores = []
    comb_score = []
    for i in range(len(review.norm_text)):
        score_list = analyzer.polarity_scores(review.norm_text[i])
        agg_score = score_list['compound']
        comb_score.append(agg_score)
        pol_scores.append('positive' if agg_score >=threshold else 'negative')
    return(pd.DataFrame({'compound score':comb_score,'sentiment':pol_scores}))
        
    

In [52]:
agg_list = sent_analyzer(review_text,.1,True)
agg_list

,compound score,sentiment
0,0.9294,positive
1,0.9989,positive
2,0.8807,positive
3,0.9829,positive
4,-0.9619,negative
...,...,...
95,0.9777,positive
96,0.9915,positive
97,0.9923,positive
98,0.9967,positive


In [22]:
analyzer.polarity_scores(norm_text[])

{'neg': 0.123, 'neu': 0.516, 'pos': 0.362, 'compound': 0.9989}

In [25]:
len(review_text.norm_text)

100